In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [13]:

# Sample text data
text = "The horse is a domesticated, one-toed, hoofed mammal. It belongs to the taxonomic family Equidae and is one of two extant subspecies of Equus ferus"
with open('/content/horse2.txt', 'r', encoding='utf-8') as file:
  plain_text = file.read()

data = """Piford Technologies is a leading Software Development Company\n
Piford Technologies provide trainings to working professionals and students\n
we are product based and service based company\n
we have one of our office in IT park,Mohali\n"""


In [14]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

# determine the vovabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)


Vocabulary Size: 23


In [15]:

# create word -> word sequences
sequences = list()
for i in range(1,len(encoded_data)):
  sequence = encoded_data[i-1:i+1]
  sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# split into X and Y elements
sequences
#sequences[:5] # [input,output]


Total Sequences: 7


[[2, 6], [6, 13], [13, 17], [17, 17], [17, 3], [3, 4], [4, 11]]

In [17]:

# Create input sequences using tokenized words
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)

# Function to generate next word
def next_word(input_text, num_next_words=1):
    for _ in range(num_next_words):
        token_list = tokenizer.texts_to_sequences([input_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        input_text += " " + output_word
    return input_text

# Generate next word
seed_text = "Horse"
next_words = 3

print(seed_text)


print(data)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 3.1347 - accuracy: 0.0800
Epoch 2/100
1/1 [==============================] - 0s 93ms/step - loss: 3.1229 - accuracy: 0.0800
Epoch 3/100
1/1 [==============================] - 0s 78ms/step - loss: 3.1107 - accuracy: 0.0800
Epoch 4/100
1/1 [==============================] - 0s 81ms/step - loss: 3.0972 - accuracy: 0.0800
Epoch 5/100
1/1 [==============================] - 0s 76ms/step - loss: 3.0813 - accuracy: 0.1200
Epoch 6/100
1/1 [==============================] - 0s 78ms/step - loss: 3.0616 - accuracy: 0.1200
Epoch 7/100
1/1 [==============================] - 0s 77ms/step - loss: 3.0363 - accuracy: 0.1200
Epoch 8/100
1/1 [==============================] - 0s 78ms/step - loss: 3.0035 - accuracy: 0.0800
Epoch 9/100
1/1 [==============================] - 0s 81ms/step - loss: 2.9635 - accuracy: 0.0800
Epoch 10/100
1/1 [==============================] - 0s 82ms/step - loss: 2.9260 - accuracy: 0.0800
Epoch 11/100
1/1 [===